In [1]:
import numpy as np
from itertools import cycle
import xml.etree.ElementTree as ET
MIN_DURATION,MAX_DURATION = 5 , 50
CYCLE_LENGTH = 7

class particule():
    def __init__(self,d):
        self.pos = np.random.uniform(MIN_DURATION,MAX_DURATION,(2,CYCLE_LENGTH))
        self.Pbest = self.pos
        self.Pbestcost = float('inf')
        self.d = d
        self.speed =np.random.uniform(-12,-12, (2,CYCLE_LENGTH))

    def setPos(self,):
        #r = 1-((self.pos.sum()/self.d))
        #temp = (self.speed > 0 ).astype(int) * r
        #temp = self.pos + self.speed
        self.pos = np.clip(self.speed + self.pos,MIN_DURATION,MAX_DURATION)

    def setSpeed(self,w,alpha,beta,Gbest):
        self.speed = np.clip(w* self.speed + alpha * np.random.rand()*(self.Pbest - self.pos) + beta * np.random.rand()*(Gbest - self.pos)\
                             ,-5,5)
        
    def setPbest(self,score):
        self.Pbest = self.pos
        self.Pbestcost = score
        


In [2]:
#!/usr/bin/env python
# Eclipse SUMO, Simulation of Urban MObility; see https://eclipse.org/sumo
# Copyright (C) 2009-2020 German Aerospace Center (DLR) and others.
# This program and the accompanying materials are made available under the
# terms of the Eclipse Public License 2.0 which is available at
# https://www.eclipse.org/legal/epl-2.0/
# This Source Code may also be made available under the following Secondary
# Licenses when the conditions for such availability set forth in the Eclipse
# Public License 2.0 are satisfied: GNU General Public License, version 2
# or later which is available at
# https://www.gnu.org/licenses/old-licenses/gpl-2.0-standalone.html
# SPDX-License-Identifier: EPL-2.0 OR GPL-2.0-or-later

# @file    runner.py
# @author  Lena Kalleske
# @author  Daniel Krajzewicz
# @author  Michael Behrisch
# @author  Jakob Erdmann
# @date    2009-03-26

from __future__ import absolute_import
from __future__ import print_function

import os
import sys
import optparse
import random

# we need to import python modules from the $SUMO_HOME/tools directory
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

from sumolib import checkBinary  # noqa
import traci  # noqa


def generate_routefile():
    random.seed(42)  # make tests reproducible
    N = 2000  # number of time steps
    # demand per second from different directions
    pWE = 1. / 10
    pEW = 1. / 12
    pNS = 1. / 50
    with open("data/cross.rou.xml", "w") as routes:
        print("""<routes>
        <vType id="typeWE" accel="0.8" decel="4.5" sigma="0.5" length="5" minGap="2.5" maxSpeed="16.67" \
guiShape="passenger"/>
        <vType id="typeNS" accel="0.8" decel="4.5" sigma="0.5" length="7" minGap="3" maxSpeed="25" guiShape="bus"/>

        <route id="right" edges="51o 1i 2o 52i" />
        <route id="left" edges="52o 2i 1o 51i" />
        <route id="down" edges="54o 4i 3o 53i" />""", file=routes)
        vehNr = 0
        for i in range(N):
            if random.uniform(0, 1) < pWE:
                print('    <vehicle id="right_%i" type="typeWE" route="right" depart="%i" />' % (
                    vehNr, i), file=routes)
                vehNr += 1
            if random.uniform(0, 1) < pNS:
                print('    <vehicle id="down_%i" type="typeNS" route="down" depart="%i" color="1,0,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
            if random.uniform(0, 1) < pEW:
                print('    <vehicle id="left_%i" type="typeWE" route="left" depart="%i" />' % (
                    vehNr, i), file=routes)
                vehNr += 1
        print("</routes>", file=routes)

# The program looks like this
#    <tlLogic id="0" type="static" programID="0" offset="0">
# the locations of the tls are      NESW
#        <phase duration="31" state="GrGr"/>
#        <phase duration="6"  state="yryr"/>
#        <phase duration="31" state="rGrG"/>
#        <phase duration="6"  state="ryry"/>
#    </tlLogic>
import traci.constants as tc
first = True
def run(particle):
    """execute the TraCI control loop"""
    step = 0
    
    network = ET.parse('data2/double.net.xml')
    nets = network.findall('tlLogic') 
    dur = [34,6,31,6] 
    global first
    j = 0
    for signal in nets:
        #print(j)
        i = 0
        for phase in signal.iter('phase'):
            #print('phase')
            #print(particle.pos.shape)
            phase.set("duration", str(int(particle.pos[j][i])))
            i+=1
            first = False
        j+=1

    network.write("data2/double.net.xml")

    #
    test = cycle([0,1,2,3])
    LastPhase = 0
    s = 0
    
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        #test+= 1
        
        #for v in traci.vehicle.getIDList():
         #   s+= .5*traci.vehicle.getWaitingTime(v) + 5*traci.vehicle.getIDCount()
        step += 1
        if step >=500:
            break
    #print(s,test)    
    traci.close()
    sys.stdout.flush()
    xmldoc = ET.parse('tripinfo2.xml')

    tripinfos = xmldoc.findall('tripinfo')

    waiting_time = 0
    for tripinfo in tripinfos:
        waiting_time += float(tripinfo.get('waitingTime'))
    return waiting_time



# this is the main entry point of this script


In [3]:
#options = get_options()
c = cycle(np.arange(10000))
    # this script has been called from the command line. It will start sumo as a
def function(particle):# server, then connect and run
    sumoBinary = checkBinary('sumo')
       # first, generate the route file for this simulation
    #generate_routefile()

        # this is the normal way of using traci. sumo is started as a
        # subprocess and then the python script connects and runs
    traci.start(['sumo', "-c", "data2/cross.sumocfg",
                                 "--tripinfo-output", "tripinfo2.xml"],port =  8873,label=f"{next(c)}")
    
    s = run(particle)
    return s
        #vehID = traci.vehicle.getIDList()[0]
        #traci.vehicle.subscribe(vehID, (tc.VAR_ROAD_ID, tc.VAR_LANEPOSITION))
    #print(print(traci.vehicle.getSubscriptionResults(vehID)))
#run(particle)

In [6]:
def pso(pop,function,maxiter = 20,alpha =.5, beta = .5 ,w = .9,):
    #initialiser les swarms
    swarms = []
    for i in range(pop):
        #initial_position = np.random.choice([0,1], CYCLE_LENGTH)
        swarms.append(particule(CYCLE_LENGTH))
        #print(swarms[i].pos)
    #initialiser Gbest
    
    costGbest = float('inf')
    Gbest = np.zeros(CYCLE_LENGTH)
    #Gfeatures = float('inf')
    history = []
    while maxiter:
        #a= np.zeros((pop,d+1))
        for i in range(pop):
            swarms[i].setSpeed(w,alpha,beta,Gbest) 
            
            swarms[i].setPos()
            temp_Res = function(swarms[i])
            if swarms[i].Pbestcost > temp_Res:
                
                swarms[i].setPbest(temp_Res)
                if costGbest > swarms[i].Pbestcost:
                    costGbest = swarms[i].Pbestcost
                    Gbest = swarms[i].Pbest
        if maxiter%1 ==0:
            print('best pos  is : ' ,Gbest)
            print('the best time is :',costGbest)
            #print(swarms[i].speed)
            #print(swarms[i].pos)
            #print(swarms[i].Pbestcost)
          
        maxiter-=1
    return costGbest,Gbest,history

In [7]:
pso(20,function)

best pos  is :  [[32.79174988 41.6150344  23.73775494  5.         26.24356565 38.41415742
  29.02309771]
 [ 5.45689883 10.22925212 39.67086246 10.71410894 18.13839484 14.52628876
   5.        ]]
the best time is : 27592.0
best pos  is :  [[30.66288823  9.45866769 27.63960511 29.66567188  5.69770221 26.92178565
  14.79954839]
 [ 7.47635169 10.72007144 37.22429557  5.         13.2726384   9.26796077
   5.        ]]
the best time is : 27495.0
best pos  is :  [[31.32627687 10.29770806 31.95069339 20.61396217 10.59785861 27.19435392
  19.35685733]
 [19.63036729  9.88574697 19.43129937  5.         13.13038836 11.8672664
   5.        ]]
the best time is : 26689.0
best pos  is :  [[24.63767089 15.53109085 23.39803209 20.16567188  8.97979609 24.21682495
  15.31323484]
 [ 8.10392551  5.         27.72429557  5.          7.63890003  5.13627158
   5.        ]]
the best time is : 26641.0
best pos  is :  [[24.63767089 15.53109085 23.39803209 20.16567188  8.97979609 24.21682495
  15.31323484]
 [ 8.103

(22894.0,
 array([[32.3214805 , 13.14841215, 13.72306191, 25.88044673, 18.66489741,
         20.36488267, 13.93733561],
        [ 5.        ,  6.63410045, 32.17336976,  5.86466016, 18.34961262,
         22.58348884, 21.81559068]]),
 [])

In [ ]:
test = [1,2,3,]
a= cycle(test)

In [ ]:
np.arange(10000)

In [ ]:
network = ET.parse('data/cross.net.xml')
signal = network.find('tlLogic') 
i = 0
for phase in signal.iter('phase'):
        print(phase.get('state'),phase.get('duration'))
        phase.set("duration", str(i))
        i+=1

#network.write("data/cross.net.xml")

In [30]:
#network = ET.parse('data/cross.net.xml')
#signal = network.find('tlLogic') 
dur = [34,6,31,6] 
i = 0
for phase in signal.iter('phase'):
    print('phase')
    phase.set("duration", str(dur[i]))
    i+=1
        
network.write("data2/double.net.xml")

phase
phase
phase
phase


In [12]:
int(5.5)

5

In [28]:
for i in signal.iter('phase'):
    print(i)

<Element 'phase' at 0x000001A493162B38>
<Element 'phase' at 0x000001A493162B88>
<Element 'phase' at 0x000001A493162BD8>
<Element 'phase' at 0x000001A493162C28>


In [25]:
np.random.uniform(MIN_DURATION,MAX_DURATION,CYCLE_LENGTH)

array([12.39787881, 26.01068221, 18.66278957, 25.58332035])

In [14]:
network = ET.parse('data2/double.net.xml')
nets = network.findall('tlLogic') 

In [17]:
i = 1
for signal in nets:
    print(signal)
    for phase in signal.iter('phase'):
        print(phase.get('duration'),phase.get('state'),)
    i+=1

<Element 'tlLogic' at 0x000001474C246D68>
17 GGGGgrrrrrGGGGgrrrrr
32 yyyyyrrrrryyyyyrrrrr
28 rrrrGrrrrrrrrrGrrrrr
17 rrrrrrrrrrrrrrrrrrrr
41 rrrrrGGGGgrrrrrGGGGg
15 rrrrryyyyyrrrrryyyyy
24 rrrrrrrrrGrrrrrrrrrG
<Element 'tlLogic' at 0x000001474C249048>
20 GGGGgrrrrrGGGGgrrrrr
33 yyyyyrrrrryyyyyrrrrr
26 rrrrGrrrrrrrrrGrrrrr
14 rrrrrGGGGgrrrrrGGGGg
28 rrrrryyyyyrrrrryyyyy
22 rrrrrrrrrGrrrrrrrrrG


In [13]:
list(signal)

[<Element 'tlLogic' at 0x0000024A389BED68>,
 <Element 'tlLogic' at 0x0000024A389B4048>]

In [9]:
network??

Type:        ElementTree
String form: <xml.etree.ElementTree.ElementTree object at 0x0000024A38A501D0>
File:        e:\anaconda3\lib\xml\etree\elementtree.py
Source:     
class ElementTree:
    """An XML element hierarchy.

    This class also provides support for serialization to and from
    standard XML.

    *element* is an optional root element node,
    *file* is an optional file handle or file name of an XML file whose
    contents will be used to initialize the tree with.

    """
    def __init__(self, element=None, file=None):
        # assert element is None or iselement(element)
        self._root = element # first node
        if file:
            self.parse(file)

    def getroot(self):
        """Return root element of this tree."""
        return self._root

    def _setroot(self, element):
        """Replace root element of this tree.

        This will discard the current contents of the tree and replace it
        with the given element.  Use with care!

        """


In [6]:
a =np.random.uniform(2,5,(2,5))

In [7]:
a

array([[4.52444726, 4.56717571, 4.36441673, 3.77274739, 3.59208039],
       [2.29134986, 2.85634027, 3.94404107, 4.2996958 , 3.71940217]])

In [11]:
a[0][3]

3.7727473932746958